In [26]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as random
from scipy.signal import convolve

%run initMethods.py
%run makeGif.py
%run methods.py

In [21]:
params = { #parameters relevant for the equations
    "Nh":            1000,
    "N0":             200,
    "R0":              20,
    "M":                1, #Also L, total number of spacers
    "D":                3, #Unused
    "mu":             0.1, #mutation rate
    "gamma_shape":     20, 
    "Np":               0, #Number of Cas Protein
    "dc":               3, #Required number of complexes to activate defence
    "h":                4, #coordination coeff
    "r":             1000, #cross-reactivity kernel
    "rho":           5e-5, #spacer sharing coeff
}
sim_params = { #parameters relevant for the simulation (including Inital Valuess)
    "xdomain":                 1000,
    "dx":                         1,
    "t0":                         0, 
    "tf":                       100,
    "dt":                         1,
    "initial_mean":           [0,0],
    "initial_var":                5,
    "n_step_prior":               5,
    "conv_size":               4000,
}

In [24]:
n = init_guassian(params["N0"], sim_params)
nh = init_guassian(params["Nh"], sim_params)

In [27]:
for i in range(5):
    f = fitness_spacers_controlled(n, nh, params, sim_params) #f is now a masked array (where mask is where eff_R0 = 0)
    n = virus_growth(n, f, params, sim_params) #update
    n = mutation(n, params, sim_params)

    nh = immunity_gain(nh, n) #update nh
    nh = immunity_loss(nh, n)

AttributeError: 'numpy.ndarray' object has no attribute 'mask'